### 2) Create a new BQ dataset to store all of your modeled tables

In [3]:
dataset_id = "datamart"

In [2]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'total-earth-236521:datamart' successfully created.


### 3) Create modeled tables by following our BigQuery design guidelines

In [11]:
%%bigquery
CREATE TABLE datamart.host AS
SELECT DISTINCT host_id, host_name, calculated_host_listings_count AS listings_count
FROM airbnb_staging.Austin_listings
UNION ALL SELECT DISTINCT host_id, host_name, calculated_host_listings_count AS listings_count FROM airbnb_staging.Boston_listings
UNION ALL SELECT DISTINCT host_id, host_name, calculated_host_listings_count AS listinsg_count FROM airbnb_staging.LA_listings

""


In [8]:
%%bigquery
CREATE TABLE datamart.location AS
SELECT DISTINCT id AS listing_id, neighbourhood_group, CAST (neighbourhood AS STRING) AS neighbourhood, latitude, longitude
FROM airbnb_staging.Austin_listings
WHERE id IS NOT NULL
UNION ALL SELECT DISTINCT id AS listing_id,  neighbourhood_group, neighbourhood, latitude, longitude
FROM airbnb_staging.Boston_listings
WHERE id IS NOT NULL
UNION ALL SELECT DISTINCT id AS listing_id,  neighbourhood_group, neighbourhood, latitude, longitude
FROM airbnb_staging.LA_listings
WHERE id IS NOT NULL

""


In [16]:
%%bigquery
CREATE TABLE datamart.apartment AS
SELECT DISTINCT id AS listing_id, IFNULL(city, "Austin") AS city, host_id, name, room_type, al.price,
al.minimum_nights, maximum_nights, availability_365, number_of_reviews, last_review, reviews_per_month
FROM airbnb_staging.Austin_listings AS al JOIN airbnb_staging.Austin_calendar AS ac
ON al.id = ac.listing_id
WHERE id IS NOT NULL
UNION ALL SELECT DISTINCT id AS listing_id, IFNULL(city, "Boston") AS city, host_id, name, room_type, bl.price,
bl.minimum_nights, maximum_nights, availability_365, number_of_reviews, last_review, reviews_per_month
FROM airbnb_staging.Boston_listings AS bl JOIN airbnb_staging.Boston_calendar AS bc
ON bl.id = bc.listing_id
WHERE id IS NOT NULL
UNION ALL SELECT DISTINCT id AS listing_id, IFNULL(city, "LA") AS city, host_id, name, room_type, ll.price,
ll.minimum_nights, maximum_nights, availability_365, number_of_reviews, last_review, reviews_per_month
FROM airbnb_staging.LA_listings AS ll JOIN airbnb_staging.LA_calendar AS lc
ON ll.id = lc.listing_id
WHERE id IS NOT NULL

""


In [17]:
%%bigquery
CREATE TABLE datamart.calendar AS 
SELECT DISTINCT listing_id, date, available 
FROM airbnb_staging.Austin_calendar
WHERE listing_id IS NOT NULL
UNION ALL SELECT DISTINCT listing_id, date, available 
FROM airbnb_staging.Boston_calendar
WHERE listing_id IS NOT NULL
UNION ALL SELECT DISTINCT listing_id, date, available 
FROM airbnb_staging.LA_calendar
WHERE listing_id IS NOT NULL

""


In [19]:
%%bigquery
CREATE TABLE datamart.review AS
SELECT DISTINCT listing_id, date
FROM airbnb_staging.Austin_reviews
UNION ALL SELECT DISTINCT listing_id, date
FROM airbnb_staging.Boston_reviews
UNION ALL SELECT DISTINCT listing_id, date
FROM airbnb_staging.LA_reviews


""


In [22]:
%%bigquery
CREATE TABLE datamart.longrent AS
SELECT DISTINCT Date as date, Austin as rental, IFNULL(city, "Austin") AS city from zillow_staging.zillow_rental
UNION ALL
SELECT DISTINCT Date as date, Boston as rental, IFNULL(city, "Boston") AS city from zillow_staging.zillow_rental
UNION ALL
SELECT DISTINCT Date as date, LA as rental, IFNULL(city, "LA") AS city from zillow_staging.zillow_rental

""


In [23]:
%%bigquery
CREATE TABLE datamart.home_value AS
SELECT DISTINCT Date as date, Austin as home_value, IFNULL(city, "Austin") AS city from zillow_staging.zillow_homevalue
UNION ALL
SELECT DISTINCT Date as date, Boston as home_value, IFNULL(city, "Boston") AS city from zillow_staging.zillow_homevalue
UNION ALL
SELECT DISTINCT Date as date, LA as home_value, IFNULL(city, "LA") AS city from zillow_staging.zillow_homevalue


""


In [24]:
%%bigquery
CREATE TABLE datamart.inventory AS
SELECT DISTINCT Date as date, Austin as inventory, IFNULL(city, "Austin") AS city from zillow_staging.zillow_inventory
UNION ALL
SELECT DISTINCT Date as date, Boston as inventory, IFNULL(city, "Boston") AS city from zillow_staging.zillow_inventory
UNION ALL
SELECT DISTINCT Date as date, LA as inventory, IFNULL(city, "LA") AS city from zillow_staging.zillow_inventory

""


In [28]:
%%bigquery 
INSERT INTO datamart.cities(city)
VALUES ("Austin")

""


In [29]:
%%bigquery 
INSERT INTO datamart.cities(city)
VALUES ("Boston")

""


In [30]:
%%bigquery 
INSERT INTO datamart.cities(city)
VALUES ("LA")

""


### 6) Explore data by writing SQL queries on modeled tables

Return the average price of each room type in each city

In [1]:
%%bigquery
SELECT avg(price) AS avg_price, city, room_type
FROM datamart.apartment
GROUP BY city, room_type
ORDER BY city, room_type

,avg_price,city,room_type
0,281.830584,Austin,Entire home/apt
1,628.636364,Austin,Hotel room
2,138.220304,Austin,Private room
3,63.437126,Austin,Shared room
4,191.143409,Boston,Entire home/apt
5,170.424242,Boston,Hotel room
6,103.286321,Boston,Private room
7,136.444444,Boston,Shared room
8,295.220401,LA,Entire home/apt
9,490.565574,LA,Hotel room


Return average price of short-term rental (minnight<10) for each room type for each city

In [4]:
%%bigquery
SELECT avg(price) AS avg_price, city, room_type
FROM datamart.apartment
WHERE minimum_nights IN (SELECT minimum_nights FROM datamart.apartment WHERE minimum_nights<10)
GROUP BY city, room_type
ORDER BY city, room_type

,avg_price,city,room_type
0,286.610357,Austin,Entire home/apt
1,628.636364,Austin,Hotel room
2,146.784921,Austin,Private room
3,84.812500,Austin,Shared room
4,211.379038,Boston,Entire home/apt
5,179.533333,Boston,Hotel room
6,161.961722,Boston,Private room
7,48.000000,Boston,Shared room
8,331.553506,LA,Entire home/apt
9,543.196172,LA,Hotel room


Return the long-term rental price (min_night>90days) for each room type for each city

In [10]:
%%bigquery
SELECT avg(price) AS avg_price, city, room_type
FROM datamart.apartment
WHERE minimum_nights IN (SELECT minimum_nights FROM datamart.apartment WHERE minimum_nights>90)
GROUP BY city, room_type
ORDER BY city, room_type

,avg_price,city,room_type
0,1295.725000,Austin,Entire home/apt
1,154.230769,Austin,Private room
2,200.541199,Boston,Entire home/apt
3,0.000000,Boston,Hotel room
4,73.251799,Boston,Private room
5,163.666667,Boston,Shared room
6,385.354037,LA,Entire home/apt
7,171.500000,LA,Hotel room
8,134.805556,LA,Private room
9,230.166667,LA,Shared room


Return the housing inventory in Boston for each month in the year 2019
that has the inventory above the average inventory of this year.

#inventory is above average -> inversely indicate the popularity of housing
or the high activity of real-estate development



In [11]:
%%bigquery
SELECT AVG(iv0.inventory) AS inventory_in_month, extract (month from iv0.date) as month
FROM datamart.inventory AS iv0
WHERE extract(year from iv0.date)=2019
AND iv0.city='Boston'
GROUP BY month
HAVING
inventory_in_month > (SELECT avg(iv1.inventory) 
FROM datamart.inventory as iv1
WHERE iv1.city='Boston' and extract(year from iv1.date)=2019)
ORDER BY inventory_in_month



,inventory_in_month,month
0,16338.0,5
1,16392.0,11
2,16959.0,9
3,16983.0,10
4,17387.0,8
5,17903.0,6
6,18267.0,7


Return the percentage increase in (monthly) Long-term rent (zillow data) over each consecutive year for each city

In [14]:
%%bigquery
SELECT 100*(rent2 - rent1)/rent1 as percent_rent_increase, rent1, rent2, year1, year2, city1
FROM
(SELECT AVG(lr1.rental) as rent1, extract (year from lr1.date) as year1, lr1.city as city1
FROM datamart.longrent as lr1
GROUP BY year1, lr1.city)
JOIN
(SELECT AVG(lr2.rental) as rent2, extract (year from lr2.date) as year2, lr2.city as city2
FROM datamart.longrent as lr2
GROUP BY year2, lr2.city)
ON year1+1=year2 and city1=city2

,percent_rent_increase,rent1,rent2,year1,year2,city1
0,5.233870,2216.333333,2332.333333,2016,2017,LA
1,4.334000,2332.333333,2433.416667,2017,2018,LA
2,-0.317170,2548.583333,2540.500000,2020,2021,LA
3,3.681381,2433.416667,2523.000000,2018,2019,LA
4,6.691271,2077.333333,2216.333333,2015,2016,LA
5,1.014004,2523.000000,2548.583333,2019,2020,LA
6,8.595077,1912.916667,2077.333333,2014,2015,LA
7,0.783016,1511.250000,1523.083333,2019,2020,Austin
8,2.691511,1408.750000,1446.666667,2017,2018,Austin
9,3.825545,1337.500000,1388.666667,2015,2016,Austin


### 7) Create data visualizations

Create BQ dataset and name it reports

In [15]:
report_dataset_id = "reports"

In [16]:
!bq --location=US mk --dataset {report_dataset_id}

Dataset 'total-earth-236521:reports' successfully created.


create a view for 2 querys in the reports dataset

In [19]:
%%bigquery
CREATE VIEW reports.City_room_price AS
SELECT avg(price) AS avg_price, city, room_type
FROM datamart.apartment
GROUP BY city, room_type
ORDER BY city, room_type

""


In [20]:
%%bigquery
CREATE VIEW reports.Percent_rent_increase AS
SELECT 100*(rent2 - rent1)/rent1 as percent_rent_increase, rent1, rent2, year1, year2, city1
FROM
(SELECT AVG(lr1.rental) as rent1, extract (year from lr1.date) as year1, lr1.city as city1
FROM datamart.longrent as lr1
GROUP BY year1, lr1.city)
JOIN
(SELECT AVG(lr2.rental) as rent2, extract (year from lr2.date) as year2, lr2.city as city2
FROM datamart.longrent as lr2
GROUP BY year2, lr2.city)
ON year1+1=year2 and city1=city2

""
